In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras import Sequential
import tensorflow_hub as hub
from tensorflow.keras.optimizers import Adam
from PIL import Image
from tqdm import tqdm
from keras.models import load_model
import cv2    
import os

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
directory = ("/content/drive/MyDrive/Colab Notebooks/flowers")

In [19]:
shape = (224, 224)

TRAINING_DATA_DIR = str(directory)

datagen_kwargs = dict(rescale=1./255, validation_split=.20)


In [20]:
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
TRAINING_DATA_DIR,
subset="validation",
shuffle=True,
target_size=shape
)

Found 860 images belonging to 5 classes.


In [21]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
train_generator = train_datagen.flow_from_directory(
TRAINING_DATA_DIR,
subset="training",
shuffle=True,
target_size=shape)

Found 3457 images belonging to 5 classes.


In [22]:
print (train_generator.class_indices)
labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open('labels.txt', 'w') as f:
 f.write(labels)


{'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflower': 3, 'tulip': 4}


In [23]:
model = tf.keras.Sequential([
 hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4", 
 output_shape=[1280],
 trainable=False),
 tf.keras.layers.Dropout(0.4),
 tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])
model.build([None, 224, 224, 3])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [24]:

optimizer = tf.keras.optimizers.Adam(lr=1e-3)
model.compile(
 optimizer=optimizer,
 loss='categorical_crossentropy',
 metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [25]:
steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
val_steps_per_epoch = np.ceil(valid_generator.samples/valid_generator.batch_size)
hist = model.fit(
 train_generator, 
 epochs=12,
 verbose=1,
 steps_per_epoch=steps_per_epoch,
 validation_data=valid_generator,
 validation_steps=val_steps_per_epoch).history

Epoch 1/12
109/109 [==============================] - 599s 5s/step - loss: 0.9188 - acc: 0.6537 - val_loss: 0.4653 - val_acc: 0.8477
Epoch 2/12
109/109 [==============================] - 26s 241ms/step - loss: 0.5071 - acc: 0.8154 - val_loss: 0.4046 - val_acc: 0.8547
Epoch 3/12
109/109 [==============================] - 26s 240ms/step - loss: 0.4247 - acc: 0.8473 - val_loss: 0.3472 - val_acc: 0.8814
Epoch 4/12
109/109 [==============================] - 26s 239ms/step - loss: 0.3755 - acc: 0.8693 - val_loss: 0.3588 - val_acc: 0.8756
Epoch 5/12
109/109 [==============================] - 26s 237ms/step - loss: 0.3360 - acc: 0.8846 - val_loss: 0.3344 - val_acc: 0.8791
Epoch 6/12
109/109 [==============================] - 26s 241ms/step - loss: 0.3146 - acc: 0.8901 - val_loss: 0.3250 - val_acc: 0.8919
Epoch 7/12
109/109 [==============================] - 26s 239ms/step - loss: 0.3091 - acc: 0.8921 - val_loss: 0.3336 - val_acc: 0.8814
Epoch 8/12
109/109 [==============================] - 26s